In [174]:
#importy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re

In [255]:
#tworze pusty dataframe
df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','powierzchnia','cena'])
df

,tytul,typ,miejsce,pokoje,powierzchnia,cena


In [133]:
#link
url = 'https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bcity_id%5D=166'
strona = requests.get(url)

In [134]:
html = strona.content

In [135]:
soup = bs(html,'lxml')

In [136]:
with open("otodom.html",'wb') as plik:
    plik.write(soup.prettify(encoding='utf-8'))

In [116]:
'''
notatki dot. tagów
- form, id="pagerForm", -> 'a',"" -> numery stron.
- <div class="offer-item-details">
    -
'''

'\nnotatki dot. tagów\n\n\n\n'

### Tworzenie listy dostępnych stron z wynikami wyszukiwania.
I obsługi limitu wyszukiwań

In [106]:
'''
pagers = soup.find('ul',"pager")
pagers'''
#nieuzywane

<ul class="pager">
<li class="pager-prev">
<a class="disabled" data-dir="previous" href=""><span class="icon-prev"></span></a>
</li>
<li><a class="active" href="https://www.otodom.pl/wynajem/mieszkanie/">1</a></li>
<li><a class="" href="https://www.otodom.pl/wynajem/mieszkanie/?page=2">2</a></li>
<li><a class="" href="https://www.otodom.pl/wynajem/mieszkanie/?page=3">3</a></li>
								...
							<li><a class="" href="https://www.otodom.pl/wynajem/mieszkanie/?page=500">500</a></li>
<li class="pager-next">
<a class="" data-dir="next" href="https://www.otodom.pl/wynajem/mieszkanie/?page=2"><span class="icon-next"></span></a>
</li>
<!--	<li class="pager-counter">
		<input id="pageParam" type="number" name="page" placeholder="500" class="input-number-noSpinners">
	</li>-->
</ul>

In [107]:
'''#znajdowanie aktualnego numeru strony
p = soup.find_all('a','active')
p
p[1].text
page_active = int(p[1].text)
page_active'''
#będzie niepotrzebne, kod poniżej tworzy listę stron.

1

In [137]:
p2 = soup.find('form', {"id":'pagerForm'})
p2= p2.find_all('a',"")

strony = []

for a in p2:
    strona = a.text
    if strona != "":
        strona = int(strona)
        strony.append(strona)
    else:
        pass
        

strona_max = max(strony)
strona_min = min(strony)

In [138]:
strona_min

1

In [139]:
strona_max

2

In [140]:
limit_wyszukiwania = 0

def limit_wyszukiwan(strona_max):
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    print(limit_wyszukiwania)

In [142]:
limit_wyszukiwan(strona_max)

2


### Tworzenie pętli do spisania wszystkich ogłoszeń z strony

In [215]:
oferty = soup.find_all('div', {"class":'offer-item-details'})

In [216]:
len(oferty)

27

In [150]:
oferty[0]

<div class="offer-item-details">
<header class="offer-item-header">
<h3>
<a data-featured-name="promo_top_ads" data-tracking="click_body" data-tracking-data='{"touch_point_button":"title"}' href="https://www.otodom.pl/pl/oferta/tychy-najpiekniejsze-reprezentacyjne-mieszkanie-ID4a4Lw.html#71b39355d2">
<strong class="visible-xs-block">42 m²</strong>
<span class="text-nowrap">
<span class="offer-item-title">TYCHY - najpiękniejsze, reprezentacyjne mieszkanie</span>
</span>
</a>
</h3>
<p class="text-nowrap"><span class="hidden-xs">Mieszkanie na wynajem: </span>Tychy, śląskie</p>
<div class="vas-list-no-offer">
<a class="button-observed observe-link favourites-button observed-text" data-id="61506945" data-statkey="ad.observed.list" href="#" rel="nofollow">
<div class="observed-text-container">
<span class="icon observed-61506945"></span>
<i class="icon-heart-filled"></i>
<div class="observed-label"></div>
</div>
</a>
</div>
</header>
<ul class="params" data-tracking="click_body" data-trackin

In [156]:
tytul = oferty[0].find('span', {"class":"offer-item-title"}).text
tytul

'TYCHY - najpiękniejsze, reprezentacyjne mieszkanie'

In [209]:
typ = oferty[0].find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
typ

'Mieszkanie na wynajem'

In [238]:
miejsce = oferty[0].find('p', {"class": "text-nowrap"}).text
miejsce = miejsce[miejsce.find(":",0,-1)+2:]
miejsce

'Tychy, śląskie'

In [172]:
pokoje = int(oferty[0].find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
pokoje

2

In [196]:
powierzchnia = int(re.findall(r'\d+',oferty[0].find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
powierzchnia

42

In [208]:
#cena = oferty[0].find('li', {'class': "offer-item-price"}).text.strip()
cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferty[0].find('li', {'class': "offer-item-price"}).text.strip()))[0])
cena

2200